In [1]:
import numpy
from time import time as rt_timer
from datetime import datetime as rt_clock
from time import sleep
import math

# Python for Lunch (PFL) - Performance Sessions - Time & Time Measurements 

# _What on Earth is wrong with Time ?_ 

This *Python for Lunch (PFL)* series shall introduce you, as the audience, to some general concepts in performance optimisation.

*Performance optimisation* in programming tries to answer the simple question: 
**How can I improve the design and code of my program so it performs its tasks as rapidly and efficiently as possible, 
given the modern hardware capabilities available to me ?**

In popular and social discussions with *technology enthusiasts*, some people of you may have heard already terms such as 
*GPU Computing*, *Vectorisation* and the use of *(Just-in-Time) C-Kernels*, in the context of improving the performance 
of (Python) programs. How could you make use of those technologies ? Also, and more importantly, does it even make sense 
to investigate how to use those technologies (i.e. techniques) for your application ? Those are the kind of questions we 
want to answer in this PFL series.

## Motivation

Before actually doing some performance optimisation, we first need to understand how we can measure the performance of 
our current program. It's like writing a scientific article: having a fancy new idea is nice, but 
you first need to investigate the current state of research before pointing out which gap is being addressed by your 
new idea. In the realm of *high-performance computing (HPC)* and software programs, performance is measured by the passing
of *time* for executing a piece of code (i.e. program runtime). As we will see today, what sounds so simple (i.e. *measuring time*) 
can actually be quite difficult, if done correctly, and is often done incorrectly outside the computer sciences.

### Time and the *reference frame*

Let's start of by doing a few little thought experiments on time. In the first case, we imagine we sit at our desk and we get a 
notification that a new MS Teams meeting is starting in 10 minutes. This notification is is fairly unambiguous: you can set an alarm
on your watch for in-10-minutes. These 10 minutes refer to the passing of our real-world time, so for a digital watch - no matter
on which device it runs, if it is networked or used my millions of people - we expect it to behave just as our manual wrist watch. In
other terms, we have an *absolute reference frame* here - which is the real-world UTC time.

Another case of time is the communication between GPS satellites and their GPS base stations: Both the base stations as well as
the satellite are running high-precision clocks and exchange to time measurements in high-precision in a satellite message. The
base stations (as: our end-user devices) receive the *time-pings* from the satellite. Base on the time difference between the 
satellite ping and the local clock, together with the base station triangulation, we can calculate the (triangular, barycentric)
position of any receiver. The time measurements in this case are in a *relative reference frame* - time passing relative to the 
satellite.

We can encounter another relative reference frame in daily life in the train: When you board the train, it shows you on the digital
dashboard the travel time from you current- to your target location. During the travel, several delays can occur, suchs as
congestion, rail change, and so forth. When you arrive at your target destimation, and you measured the real-world time passed
for the travel, then the latter measurement will be larger than what was initially displayed. Now, the question is: what was the
actual travel time ? Was it the real-world time passing between you boarding the train and you arriving, or the accumulated time(steps)
the train was actually _moving_ on rails toward the destination ? This latter measurement is also a relative reference frame - a
time measurement that is relative to the motion of the train. And this case is also similar to what we encounter in computing when
measuring time.

### Our introduction example: How to measure 100ms on a computer ?

As a starting point in measuring time, we start off with a simple task:

**Measure the passing of a 100ms during code execution**

This task seems mundane, perhaps very abstract, as you can say: well, I just start a counter, put my program to sleep 
for 0.1 second, and then take the counter again - and 100ms have passed! 

Well, this is _not_ the case. It's the difference between *defining* PI in a program via its calculator value (i.e. 3.141592654) 
and *calculating* PI via Monte Carlo sampling.

The following function code measure the passing of 100ms using two (seamingly different) timing modes, by doing the following:
- run a global loop to measure 10 sec (to be resilient to measurement outliers)
- within that global loop, obtain a start time *t(0)*
- check if the difference between the current time *t(x)* and the start time *t(0)* is greater-or-equal-to 0.1s
- if that is the case, the current time becomes the end measurement *t(n)*
- we compute *delta(t(n), t(0))*, which is then ideally exactly 0.1s
- for the next measurement cycle, we set *t(0) = t(n)*
- for statistical evaluation, we log *delta(t(n), t(0))*
- if *t(x) - t(0) < 0.1s*, we do nothing (i.e. we wait for a fraction of 100ms before checking again)

The expected outcome of this test:
- *delta(t(n), t(0))* is always exactly 0.1s
- *delta(t(n), t(0))* is always the same, for both timing modes and all individual 100ms-intervals

In [2]:
def measure_time_simple(sync_bit=None, print_each=False):
    current_time = rt_timer()
    stime_rt_global = rt_timer()
    etime_rt_global = rt_timer()
    sclock_rt_global = rt_clock.now()
    eclock_rt_global = rt_clock.now()
    rt_global_stime = current_time
    rt_clock_deltas = []
    rt_time_deltas = []
    
    printed = False
    while (current_time-rt_global_stime) < 10.0:
        current_time = rt_timer()
        if (current_time-stime_rt_global) >= 0.1:
            etime_rt_global = rt_timer()
            eclock_rt_global = rt_clock.now()
            dt_rt_timer_global = etime_rt_global-stime_rt_global
            dt_rt_clock_global = (eclock_rt_global-sclock_rt_global).total_seconds()
            if print_each or not printed:
                print("--------------------------------------------------------------")
                print("'dt' via real-world global timer: {:10.7f}".format(dt_rt_timer_global))
                print("'dt' via real-world global clock: {:10.7f}".format(dt_rt_clock_global))
                # print("System time clock (real-world time): {}".format(eclock_rt_global))
                print("abs_delta(timer, clock): {:10.7f}".format(math.fabs(dt_rt_timer_global-dt_rt_clock_global)))
            rt_time_deltas.append( math.fabs(dt_rt_timer_global-0.1) )
            rt_clock_deltas.append( math.fabs(dt_rt_clock_global-0.1) )
            if print_each or not printed:
                print("abs_delta(timer, definitive dt=100ms): {:10.7f}".format(rt_time_deltas[-1]))
                print("abs_delta(clock, definitive dt=100ms): {:10.7f}".format(rt_clock_deltas[-1]))
                print("--------------------------------------------------------------")
            stime_rt_global = etime_rt_global
            sclock_rt_global = eclock_rt_global
            printed = True
        sleep(0.02)
    
    time_deltas = numpy.array(rt_time_deltas)
    clock_deltas = numpy.array(rt_clock_deltas)
    print("Statistical comparison:")
    print("\ttime-deltas [s]")
    print("\t\tmin: {:10.7f}".format(time_deltas.min()))
    print("\t\tmax: {:10.7f}".format(time_deltas.max()))
    print("\t\tavg: {:10.7f}".format(time_deltas.mean()))
    print("\t\tstd_dev: {:10.7f}".format(time_deltas.std()))
    print("\tclock-deltas [s]")
    print("\t\tmin: {:10.7f}".format(clock_deltas.min()))
    print("\t\tmax: {:10.7f}".format(clock_deltas.max()))
    print("\t\tavg: {:10.7f}".format(clock_deltas.mean()))
    print("\t\tstd_dev: {:10.7f}".format(clock_deltas.std()))
    if sync_bit is not None:
        sync_bit.set()
    return True

result = measure_time_simple()

--------------------------------------------------------------
'dt' via real-world global timer:  0.1004894
'dt' via real-world global clock:  0.1004910
abs_delta(timer, clock):  0.0000016
abs_delta(timer, definitive dt=100ms):  0.0004894
abs_delta(clock, definitive dt=100ms):  0.0004910
--------------------------------------------------------------
Statistical comparison:
	time-deltas [s]
		min:  0.0003866
		max:  0.0019161
		avg:  0.0009606
		std_dev:  0.0003880
	clock-deltas [s]
		min:  0.0003900
		max:  0.0019160
		avg:  0.0009606
		std_dev:  0.0003880


Looking at the results, we can conclude the following: definitive timing of 100 ms (or less) not attainable with the 
method we used (i.e. the clocks and time modules). This may be because they have a too small resolution to be applicable 
to our case. If this hythesis holds true, the above-used timing tools *time.time()* and *datetime.datetime.now()* provide 
fraudulent timings when doing performance measurements.

In order to prove the hypothesis, we need further testing. For that, we assess the influence of *concurrent processes* 
(i.e. other processes running simultaneously on the computer aside the time simulation). The expected result is as follows:

- if the timing stays the same while having other background processes running, the time modules are good - they may just lack resolution
- if the timing differs with a larger amount of background processes, we are not measuring 100ms in our program, but we're measuring 100ms system-wide - which leads to fraudulent measurements, as there are always background programs running on any digital computing device

In the example, in order to assess this, we spawn a large number of concurrent processes that do nothing else in the 
background that counting up a sum and wait for a small fraction of time.

In [3]:
from multiprocessing import Process, Event
from numpy.random import default_rng
from scipy import signal

ext_delay = 0.0075
def external_work(sync_bit):
    a = 1
    is_done = False
    while not is_done:
        sleep(ext_delay)
        a += a
        is_done = sync_bit.is_set()
        

sync_bit = Event()
concurrent_processes = [Process(target=external_work, args=(sync_bit,)) for i in range(0, 300)]
for proc in concurrent_processes:
    proc.start()
    
result = measure_time_simple(sync_bit)
    
for proc in concurrent_processes:
    proc.join()

--------------------------------------------------------------
'dt' via real-world global timer:  0.1041305
'dt' via real-world global clock:  0.1041240
abs_delta(timer, clock):  0.0000065
abs_delta(timer, definitive dt=100ms):  0.0041305
abs_delta(clock, definitive dt=100ms):  0.0041240
--------------------------------------------------------------
Statistical comparison:
	time-deltas [s]
		min:  0.0002734
		max:  0.0107309
		avg:  0.0011540
		std_dev:  0.0017378
	clock-deltas [s]
		min:  0.0002740
		max:  0.0107330
		avg:  0.0011540
		std_dev:  0.0017375


What we can see from the results is that the average deviation between our target measurement (i.e. *100ms*) and our 
actual measurement grows larger, as does the standard deviation of the measurement error. In conclusion, we can say that 
with other processes running concurrently, accurate timming is becoming increasingly difficult - if not even impossible.

Remember: in this example, we attempt to measure **100 ms** time steps, whereas in a realistic setup with hundreds of 
concurrent processes running, we measure on average time steps of **100 +/- 38 ms** - a measurement dilution of 38 percent!

Why ? What is the culprit ? And: do we need to care ?

### A physical example of performance measurement - particle tracing on flow fields

Let's first design a physical example, to see if those deviations we saw in the abstract timing example actually play a 
role when estimating the performance of physics simulations. As a illustration, let's consider a simplified particle 
tracing method on flow fields.

In [4]:
from scipy.io import netcdf
from scipy.interpolate import interpn

class Particle():
    
    def __init__(self, x, y, keep_trace_history=False):
        self._pt = numpy.array([x, y, .0])
        self._trace_history = None
        if keep_trace_history:
            self._trace_history = []

    def __del__(self):
        del self._pt
        if self._trace_history is not None:
            del self._trace_history[:]

    def advect(self, u_array, v_array, griddims, dt, fdt):
        if self._trace_history is not None and len(self._trace_history)<1:
            self._trace_history.append(numpy.array(self._pt))
        lon_range = griddims[2][-1] - griddims[2][0]
        lat_range = griddims[1][-1] - griddims[1][0]
        pt = numpy.transpose(self._pt, (2,1,0))
        uv = numpy.array([interpn(griddims, u_array, pt, method='linear', fill_value=.0), interpn(griddims, v_array, pt, method='linear', fill_value=.0)])
        self._pt[0:1] += ((uv*dt) /1000.0) # particles live on a km grid
        while self._pt[0] > griddims[2][-1]:
            self._pt[0] -= lon_range
        while self._pt[0] < griddims[2][0]:
            self._pt[0] += lon_range

        while self._pt[1] > griddims[1][-1]:
            self._pt[1] -= lat_range
        while self._pt[1] < griddims[1][0]:
            self._pt[1] += lat_range
        self._pt[2] += dt
        if self._trace_history is not None:
            self._trace_history.append(numpy.array(self._pt))

    def advect_uv(self, u, v, griddims, dt, fdt):
        if self._trace_history is not None and len(self._trace_history)<1:
            self._trace_history.append(numpy.array(self._pt))
        self._pt[0] = self._pt[0] + ((u*dt) /1000.0) # particles live on a km grid
        self._pt[1] = self._pt[1] + ((v*dt) /1000.0) # particles live on a km grid
        # -- boundary condition treatment -- #
        if self._pt[0] > griddims[2][-1]:
            self._pt[0] = griddims[2][-1]
        if self._pt[0] < griddims[2][0]:
            self._pt[0] = griddims[2][0]
            
        if self._pt[1] > griddims[1][-1]:
            self._pt[1] = griddims[1][-1]
        if self._pt[1] < griddims[1][0]:
            self._pt[1] = griddims[1][0]

        self._pt[2] += dt
        # -- history tracing - only applicable for plotting the particle trajectories -- #
        if self._trace_history is not None:
            self._trace_history.append(numpy.array([self._pt[0], self._pt[1]]))
    
    @property
    def pt(self):
        return self._pt
    
    def point(self, tstep):
        if self._trace_history is not None:
            return self._trace_history[tstep]
        return self._pt

    def time_index(self, ft):
        # expect ft to be forward-linear
        f_dt = ft[1]-ft[0]
        f_interp = self._pt[2] / f_dt
        ti = int(math.floor(f_interp))
        return ti

    def time_partion(self, ft):
        # expect ft to be forward-linear
        f_dt = ft[1]-ft[0]
        f_interp = self._pt[2] / f_dt
        f_t = f_interp-math.floor(f_interp)
        return f_t


In [6]:
class Simulation():
    def __init__(self, fx, fy, ft, fu, fv, num_visual_traces=0):
        self._data = []
        self._fx = fx
        self._fy = fy
        self._ft = ft
        self._fu = fu
        self._fv = fv
        self._gdims = (self._ft, self._fy, self._fx)
        self._simtime = .0
        self._num_visual_traces = num_visual_traces
        
    def __del__(self):
        del self._data[:]

    def __getitem__(self, item):
        max_item = len(self._data)-1
        return self._data[min(item, max_item)]
    
    @property
    def sim_time(self):
        return self._simtime
    
    @property
    def sim_end_time(self):
        return self._ft[-1]

    def add_particle(self, x, y):
        keep_trace_history = False
        if len(self._data) < self._num_visual_traces:
            keep_trace_history = True
        self._data.append(Particle(x, y, keep_trace_history))
        
    def advect_once(self, dt):
        fdt = self._ft[1]-self._ft[0]
        x = []
        y = []
        t = []
        for p in self._data:
            x.append(p.pt[0])
            y.append(p.pt[1])
            t.append(p.pt[2])
        pts = (numpy.array(t), numpy.array(y), numpy.array(x))
        us = interpn(self._gdims, self._fu, pts, method='linear', fill_value=.0)
        vs = interpn(self._gdims, self._fv, pts, method='linear', fill_value=.0)
        for i, p in enumerate(self._data):
            #p.advect(self._fu, self._fv, self._gdims, dt, fdt)
            p.advect_uv(us[i], vs[i], self._gdims, dt, fdt)
        self._simtime += dt

    def time_index(self):
        if len(self._data) < 1:
            return 0
        return self._data[0].time_index(self._ft)

    def time_partion(self):
        if len(self._data) < 1:
            return 0
        return self._data[0].time_partion(self._ft)
    
    def time_index_value(self, tx):
        # expect ft to be forward-linear
        f_dt = self._ft[1]-self._ft[0]
        f_interp = tx / f_dt
        ti = int(math.floor(f_interp))
        return ti

    def time_partion_value(self, tx):
        # expect ft to be forward-linear
        f_dt = self._ft[1]-self._ft[0]
        f_interp = tx / f_dt
        f_t = f_interp-math.floor(f_interp)
        return f_t


The *Particle* and *Simulation* class are the core elements of our simulation. In true lagrangian manner though,
we also need some flow field data on which those particles are being advected on. Those are loaded from prepared 
temporal-2D perlin-noise field sample.

In [1]:
import os, requests

if not os.path.exists('perlin.nc'):
    data_url = "https://surfdrive.surf.nl/files/index.php/s/T7QyLbGjaGMdnVD/download"
    requests.get(data_url)
# ==== Load flow-field data ==== #
f = netcdf.netcdf_file('perlin.nc', 'r')
ftimes = f.variables['time'].data
fX = f.variables['lon'].data
fY = f.variables['lat'].data
fU = f.variables['u'].data
fV = f.variables['v'].data
f.close()

NameError: name 'netcdf' is not defined

After now having prepared the simulation and obtained its field data, we can actually create a specific simulation. 

In [ ]:
sim = Simulation(fX, fY, ftimes, fU, fV)


Let us now say: we want to simulate 5000 particles with an update interval *dt=12 hours*, and measure the time it 
takes to (a) advect the particles in each simulation step (i.e. kernel runtime) and (b) complete the simulation as a 
whole (i.e total runtime).

In [7]:
a = numpy.array([fX[0], fY[0]])
b = numpy.array([fX[-1], fY[-1]])
t_0_N = numpy.array([ftimes[0], ftimes[-1]])
T = t_0_N[1]-t_0_N[0]
dt = 12.0 * 60.0 * 60.0
sim_steps = int(math.floor(T/dt))

for i in range(0, 5000):
    pt = (b - a) * numpy.random.random_sample(2)  + a
    sim.add_particle(pt[0], pt[1])

rt_times_kernel = []

print("==== running JUST the simulation ====")
stime_rt_sim = rt_timer()
prev_int_proc = -1
cur_int_proc = 0
while sim.sim_time < sim.sim_end_time:
    stime_rt_kernel = rt_timer()
    sim.advect_once(dt)
    etime_rt_kernel = rt_timer()
    rt_times_kernel.append(etime_rt_kernel-stime_rt_kernel)
    proc_done = (sim.sim_time / sim.sim_end_time)*100
    cur_int_proc = int(math.floor(proc_done/10))
    if prev_int_proc != cur_int_proc:
        print("Processing at {:10.7f} percent ...".format(proc_done))
    prev_int_proc = cur_int_proc

etime_rt_sim = rt_timer()
print("Total real-time runtime: {:10.7f} sec.".format(etime_rt_sim-stime_rt_sim))
avg = numpy.array(rt_times_kernel).mean()
print("Average real-time kernel runtime: {:10.7f} sec.".format(avg))
del sim

==== running JUST the simulation ====
Processing at  0.1351351 percent ...
Processing at 10.0000000 percent ...
Processing at 20.0000000 percent ...
Processing at 30.0000000 percent ...
Processing at 40.0000000 percent ...
Processing at 50.0000000 percent ...
Processing at 60.0000000 percent ...
Processing at 70.0000000 percent ...
Processing at 80.0000000 percent ...
Processing at 90.0000000 percent ...
Processing at 100.0000000 percent ...
Total real-time runtime: 10.1942875 sec.
Average real-time kernel runtime:  0.0137104 sec.


This simulation alone tells us very little about the performance. As we need a comparison figure, we proceed as we
did before with the 100ms-time measurement: we run the simulation and measure the *processing runtime* during the 
execution of many other, competing processes. This will allow us to validate if we see similar timing distortions with
the physical simulation as we did before with the abstract simulation.

In [8]:
sim_interference_delay = 0.001
def sim_interference(sync_bit):
    is_done = False
    
    rng = default_rng()
    matrix = rng.standard_normal(size=(64, 64))
    kernel = rng.standard_normal(size=(3, 3)) 
    while not is_done:
        # temp = numpy.convolve(matrix, kernel, mode='same')
        temp = signal.convolve2d(matrix, kernel, boundary='symm', mode='same')
        matrix = temp
        sleep(sim_interference_delay)
        is_done = sync_bit.is_set()

write_anim = False
sim = None
if write_anim:
    sim = Simulation(fX, fY, ftimes, fU, fV, num_visual_traces=25)
else:
    sim = Simulation(fX, fY, ftimes, fU, fV)

for i in range(0, 5000):
    pt = (b - a) * numpy.random.random_sample(2)  + a
    sim.add_particle(pt[0], pt[1])

rt_times_kernel.clear()
sync_bit.clear()
concurrent_processes = [Process(target=sim_interference, args=(sync_bit,)) for i in range(0, 100)]
for proc in concurrent_processes:
    proc.start()

print("==== running the simulation with competing procs ====")
stime_rt_sim = rt_timer()
prev_int_proc = -1
cur_int_proc = 0
while sim.sim_time < sim.sim_end_time:
    stime_rt_kernel = rt_timer()
    sim.advect_once(dt)
    etime_rt_kernel = rt_timer()
    rt_times_kernel.append(etime_rt_kernel-stime_rt_kernel)
    proc_done = (sim.sim_time / sim.sim_end_time)*100
    cur_int_proc = int(math.floor(proc_done/10))
    if prev_int_proc != cur_int_proc:
        print("Processing at {:10.7f} percent ...".format(proc_done))
    prev_int_proc = cur_int_proc

sync_bit.set()
etime_rt_sim = rt_timer()
print("Total real-time runtime: {:10.7f} sec.".format(etime_rt_sim-stime_rt_sim))
avg = numpy.array(rt_times_kernel).mean()
print("Average real-time kernel runtime: {:10.7f} sec.".format(avg))


for proc in concurrent_processes:
    proc.join()

==== running the simulation with competing procs ====
Processing at  0.1351351 percent ...
Processing at 10.0000000 percent ...
Processing at 20.0000000 percent ...
Processing at 30.0000000 percent ...
Processing at 40.0000000 percent ...
Processing at 50.0000000 percent ...
Processing at 60.0000000 percent ...
Processing at 70.0000000 percent ...
Processing at 80.0000000 percent ...
Processing at 90.0000000 percent ...
Processing at 100.0000000 percent ...
Total real-time runtime: 110.0267024 sec.
Average real-time kernel runtime:  0.1476274 sec.


Let's first visualise the simulation for visual verification before discussing the timing results.

In [9]:
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation, writers
from DecayLine import DecayLine


if write_anim:
    fig, ax = plt.subplots(1, 1)
    cs = ax.pcolormesh(fX, fY, fU[0], shading='gouraud', zorder=1)
    cs.cmap.set_over('k')
    cs.cmap.set_under('w')
    ax.set_title("Simulation t = %10.4f s"% (0))
    lines = []
    for i in range(0, 25):
        lines.append(DecayLine(14, 8, [1.0, 0.625, 0.], zorder=2+i))
    
    for i in range(0, 25):
        lpt = sim[i].pt
        lines[i].add_point(lpt[0], lpt[1])
    
    for l in lines:
        ax.add_collection(l.get_LineCollection())
    plt.show()

    def init():
        cs.set_array(fU[0].ravel())
        
        results = [cs, ]
        for l in lines:
            results.append(l.get_LineCollection())
    
        ax.set_title("Simulation t = %10.4f s"% (0.))
        return results
    
    def update_flow_only(frames, *args):
        if (frames % 100) == 0:
            print("Plotting frame {} ...".format(frames))
        dt = args[0]
        tx = float(frames)*dt
        tx = math.fmod(tx, ftimes[-1])
        ti0 = sim.time_index_value(tx)
        tt = sim.time_partion_value(tx)
        ti1 = 0
        if ti0 < (len(ftimes)-1):
            ti1 = ti0+1
        else:
            ti1 = 0
        fu_show = tt*fU[ti0] + (1.0-tt)*fU[ti1]
        cs.set_array(fu_show.ravel())
        
        if frames>0:
            for pindex in range(0, 25):
                pt = sim[pindex].point(frames)
                lines[pindex].add_point(pt[0], pt[1])
        
        results = [cs, ]
        for l in lines:
            results.append(l.get_LineCollection())
    
        ax.set_title("Simulation t = %10.4f s"% (tx))
        return results
    
    #Writer = writers['ffmpeg']
    #ani_writer = Writer(fps=25, bitrate=24000)
    Writer = writers['imagemagick_file']
    ani_writer = Writer()
    
    #plt.show()
    #ani = FuncAnimation(fig, update_flow_only, init_func=init, frames=sim_steps, interval=25, fargs=[dt,], blit=True)
    #ani.save("flow_w_particles.mp4", writer=ani_writer)

    ani = FuncAnimation(fig, update_flow_only, init_func=init, frames=sim_steps, fargs=[dt,], blit=True)
    ani.save("video_imgs/flow_w_particles.png", writer=ani_writer, dpi=150)


We observe a similar (if not even more drastic) timing deviation as before: When competing, simulation-unrelated 
background processes are running, the measured *processing runtime* of our simulation increases _without having made any 
modification to the simulation code itself*.

Million-dollar question: Does the simulation __itself__ really take many times longer ? After all, we haven't done anything to the simulation that could cause it to run longer, or have we ?

Billion-dollar question: Is it possible that we are measuring **time** itself incorrectly ?

Answering these questions is important to (a) *measure* the performance of a program, process or simulation, 
to (b) *locate* actual performance bottlenecks within the simulation-in-question and 
to (c) *improve* the performance, efficiency and scalability of a simulation by addressing the _actual_ performance bottlenecks.

It's also important if your actual goal is to  get academic recognition (i.t.o. published papers) from your peers in the computer-scientific domain that actually
cares about performance and efficiency - namely *High-Performance Computing (HPC)*. A paper that doesn't know which *time* it measures
or that measures the *incorrect time*, is not getting published or cited.

So, where does this discrepancy in time measurements actually originate ?

In [10]:
del sim

=> **Lecture slides**

#### Take-away messages
- real-world time /neq processing time
- processing time /neq compute time

processing time = turnaround time = the time that *your process* spends doing *something* between 2 time measurements

processing time consists of:
- (external) I/O waiting time -> time spend by your process to access files, databases or network streams
- (internal) I/O waiting time = memory I/O -> time spend copying *data* between different locations in *memory* -> quite a large chunk of time spend in Python
- draw-out waiting time -> plotting time
- compute time = *service time* -> the actual time spent in calculating stuff

Now, before going into detail what those partial times are telling you, let's check where we are.

We observed that **(a)** measuring a specific time frame (e.g. 100 ms) isn't done accurately by measuring the global time passed in the real-world during the measurement,
and that **(b)** the measured time it takes to perform a simulation varies with respect to other competing processes active in a given computing system (i.e. *a machine*).

Our major questions where:
- Why do those time measurements differ ? 

*Theory*: we measure time incorrectly, or: we measure the 'wrong' time.
- What is the culprit ?

*Answer*: process interference from concurrently running processes, that result in the fact that the procedure or process we want to measure does not run at each and every moment - it needs to share its processing time with those other cocurrent process, which is managed by the *process scheduler* of the *operating system*, and thus: out of our control.
- Do we need to care about the measurement differences ?

*Answer*: We need to care if *(a)* our calculation depends on measuring the time passed between two instances or *(b)* if we want to measure how long it takes to execute a simulation, thus making _educated_ statements about computational bottlenecks and prioritise performance improvements sensefully. 
- *How do we make sure that we measure time the way we intend it to do ?* -> to come now ...

## Computers, Python, Clocks and Time - Different clocks for different demands and situations

The computer, the operating system and Python (as the programmable interface for practitioners) provide different clocks for addressing the different situations.

### Real-world clocks

In order two measure the time passed between two events in the real world, on a global level as seen from a computer-perspective, Python provides ous with two packages we already know.

In [11]:
from time import time as rt_timer
from datetime import datetime as rt_clock

def time_globally():
    stime_rt_global = rt_timer()
    sclock_rt_global = rt_clock.now()
    # ----------------------------------------------------- #
    # -- perform something here - or just: let time pass -- #
    # ----------------------------------------------------- #
    etime_rt_global = rt_timer()
    eclock_rt_global = rt_clock.now()
    # -- times passed in real-world clocks.     -- #
    # -- It will be equal to you measuring time -- #
    # -- on a manual clock simultaneously.      -- #
    delta_t_time_func = etime_rt_global-stime_rt_global
    delta_t_clock_func = eclock_rt_global-sclock_rt_global

While taking this global, real-world equivalent clock to measure time, it is incorrect in 99.99999 percent of the cases that involve computerised procedures.

### Global computational clocks with a high resolution (~ 1-0.1 ns)

There exists a more high-resolution global clock that is not bound to the low-resolution system clock of the operating system, but on the clocks of the CPU. Python gives access to that clock via:

In [12]:
from time import perf_counter as rt_timer_hr

def compute_time_ptimer_hr():
    stime_ptimer_hr = compute_ptimer_hr()
    # --------------------------- #
    # -- perform a calculation -- #
    # --------------------------- #
    etime_ptimer_hr = compute_ptimer_hr()
    delta_t_ptimer_hr_func = etime_ptimer_hr-stime_ptimer_hr

### Process-owned computational clock with a low resolution (~1 sec. to 1/10th sec.)

If we are actually interested in finding bottlenecks in a computation within a process, a low-resolution clock (based on the system clock of the operating system) that can avoid numeric overflows in the time measurement for long time spans can be accessed. The natural drawback
for expressing long time spans and thus high-integer numbers is that the clocks are limited in their resolution. Python provides us specifically with the following packages:

In [13]:
# from timeit import timeit
from timeit import timeit

def compute_time_via_timeit():
    # --------------------------- #
    # -- perform a calculation -- #
    # --------------------------- #
    i = 0
    return True

timeit("compute_time_via_timeit()", setup="from __main__ import compute_time_via_timeit", number=10)


    

2.274000053148484e-06

### Process-owned computational clock with a high-resolution (~ 1-0.1 ns)

If we are actually interested in finding bottlenecks in a computation within a process that account 
for the scheduling- and process concurrency error _on a single machine_, then the operating system and Python provide us with
process-bound high resolution clocks. Due to the supreme processing speed of any modern computer,
individual functions are executed within just a few 100 (<500) clock ticks of the CPU - example: our per-particle kernel. 
To put this number in perspective:

A modern computer has an adaptive clock speed. If I measure this on my local machine (-> Ubuntu laptop), the clock speed ranges between

400 MHz (idle mode) - 4600 MHz (when doing calculations)

At that speed, each clock tick of the CPU takes 217 ns. Hence, calculating a particle kernel for a single particle requires at max. ~108 ps. This means three things:
- (i)   a low-resolution clock is useless for measuring processing speed and performance.
- (ii)  a resolution below 1 ns is none-sense - meaning also that we cannot measure any phenomenon with *common computers* that takes less than 1 ns
- (iii) such a high-resolution clock is based on counting CPU clocks themselves. This means that we do get a very high resolution, but it also means that we can only measure times between 10^-12 sec. to 10^7 sec. with this clock. Thus: we cannot measure long-running procedures with this clock.

Now, how does Python allow us to use this clock ?

In [14]:
from time import process_time as compute_ptimer_hr

def compute_time_ptimer_lr():
    stime_ptimer_lr = compute_ptimer_hr()
    # --------------------------- #
    # -- perform a calculation -- #
    # --------------------------- #
    etime_ptimer_lr = compute_ptimer_hr()
    delta_t_ptimer_lr_func = etime_ptimer_lr-stime_ptimer_lr


## Bringing it all together - A - Measure 100 ms

Now, lets measure the 100ms-gap with the different timer means during process-concurrency interference. First, we define the timing functions.

In [15]:
def measure_time_globally(sync_bit=None, print_each=False):
    current_time = rt_timer()
    stime_rt_global = rt_timer()
    etime_rt_global = rt_timer()
    rt_global_stime = current_time
    rt_time_deltas = []
    
    printed = False
    while (current_time-rt_global_stime) < 1.0:
        current_time = rt_timer()
        if (current_time-stime_rt_global) >= 0.1:
            etime_rt_global = rt_timer()
            dt_rt_timer_global = etime_rt_global-stime_rt_global
            if print_each or not printed:
                print("--------------------------------------------------------------")
                print("'dt' via real-world global timer: {:10.7f}".format(dt_rt_timer_global))
            rt_time_deltas.append( math.fabs(dt_rt_timer_global-0.1) )
            if print_each or not printed:
                print("abs_delta(timer, definitive dt=100ms): {:10.7f}".format(rt_time_deltas[-1]))
                print("--------------------------------------------------------------")
            stime_rt_global = etime_rt_global
            printed = True
        sleep(0.02)
    
    time_deltas = numpy.array(rt_time_deltas)
    if sync_bit is not None:
        sync_bit.set()
    return time_deltas

def measure_global_time_hr(sync_bit=None, print_each=False):
    current_time = rt_timer_hr()
    stime_rt_global = rt_timer_hr()
    etime_rt_global = rt_timer_hr()
    rt_global_stime = current_time
    rt_time_deltas = []
    
    printed = False
    while (current_time-rt_global_stime) < 1.0:
        current_time = rt_timer_hr()
        if (current_time-stime_rt_global) >= 0.1:
            etime_rt_global = rt_timer_hr()
            dt_rt_timer_global = etime_rt_global-stime_rt_global
            if print_each or not printed:
                print("--------------------------------------------------------------")
                print("'dt' via low-resolution process timer: {:10.7f}".format(dt_rt_timer_global))
            rt_time_deltas.append( math.fabs(dt_rt_timer_global-0.1) )
            if print_each or not printed:
                print("abs_delta(timer, definitive dt=100ms): {:10.7f}".format(rt_time_deltas[-1]))
                print("--------------------------------------------------------------")
            stime_rt_global = etime_rt_global
            printed = True
        sleep(0.02)
    
    time_deltas = numpy.array(rt_time_deltas)
    if sync_bit is not None:
        sync_bit.set()
    return time_deltas

def measure_compute_time_ptimer_hr(sync_bit=None, print_each=False):
    current_time = compute_ptimer_hr()
    stime_rt_global = compute_ptimer_hr()
    etime_rt_global = compute_ptimer_hr()
    rt_global_stime = current_time
    rt_time_deltas = []
    
    printed = False
    while (current_time-rt_global_stime) < 1.0:
        current_time = compute_ptimer_hr()
        if (current_time-stime_rt_global) >= 0.1:
            etime_rt_global = compute_ptimer_hr()
            dt_rt_timer_global = etime_rt_global-stime_rt_global
            if print_each or not printed:
                print("--------------------------------------------------------------")
                print("'dt' via high-resolution process timer: {:10.7f}".format(dt_rt_timer_global))
            rt_time_deltas.append( math.fabs(dt_rt_timer_global-0.1) )
            if print_each or not printed:
                print("abs_delta(timer, definitive dt=100ms): {:10.7f}".format(rt_time_deltas[-1]))
                print("--------------------------------------------------------------")
            stime_rt_global = etime_rt_global
            printed = True
        sleep(0.02)
    
    time_deltas = numpy.array(rt_time_deltas)
    if sync_bit is not None:
        sync_bit.set()
    return time_deltas

Now, we start the simulation.


In [16]:
from multiprocessing import Process, Event
nsync = Event()
concurrent_processes = [Process(target=external_work, args=(nsync,)) for i in range(0, 200)]
for proc in concurrent_processes:
    proc.start()

rt_start = rt_timer()
global_time_dts = measure_time_globally()
rt_end = rt_timer()
print("Measuring 1.0s in 0.1s steps in real-world time via the global timer took {} sec.".format(rt_end-rt_start))
rt_start = rt_timer()
global_hr_time_dts = measure_global_time_hr()
rt_end = rt_timer()
print("Measuring 1.0s in 0.1s steps in real-world time via the high-resolution global timer took {} sec.".format(rt_end-rt_start))
rt_start = rt_timer()
ptimer_hr_time_dts = measure_compute_time_ptimer_hr()
rt_end = rt_timer()
print("Measuring 1.0s in 0.1s steps in real-world time via the high-resolution process timer took {} sec.".format(rt_end-rt_start))
nsync.set()

for proc in concurrent_processes:
    proc.join()

print("Statistical comparison:")
print("\ttime-deltas - global timer [s]")
print("\t\tmin: {:10.7f}".format(global_time_dts.min()))
print("\t\tmax: {:10.7f}".format(global_time_dts.max()))
print("\t\tavg: {:10.7f}".format(global_time_dts.mean()))
print("\t\tstd_dev: {:10.7f}".format(global_time_dts.std()))
print("\ttime-deltas - high-resolution global timer [s]")
print("\t\tmin: {:10.7f}".format(global_hr_time_dts.min()))
print("\t\tmax: {:10.7f}".format(global_hr_time_dts.max()))
print("\t\tavg: {:10.7f}".format(global_hr_time_dts.mean()))
print("\t\tstd_dev: {:10.7f}".format(global_hr_time_dts.std()))
print("\ttime-deltas - high-resolution process timer [s]")
print("\t\tmin: {:10.7f}".format(ptimer_hr_time_dts.min()))
print("\t\tmax: {:10.7f}".format(ptimer_hr_time_dts.max()))
print("\t\tavg: {:10.7f}".format(ptimer_hr_time_dts.mean()))
print("\t\tstd_dev: {:10.7f}".format(ptimer_hr_time_dts.std()))


--------------------------------------------------------------
'dt' via real-world global timer:  0.1003423
abs_delta(timer, definitive dt=100ms):  0.0003423
--------------------------------------------------------------
Measuring 1.0s in 0.1s steps in real-world time via the global timer took 1.0212059020996094 sec.
--------------------------------------------------------------
'dt' via low-resolution process timer:  0.1003890
abs_delta(timer, definitive dt=100ms):  0.0003890
--------------------------------------------------------------
Measuring 1.0s in 0.1s steps in real-world time via the high-resolution global timer took 1.0346643924713135 sec.
--------------------------------------------------------------
'dt' via high-resolution process timer:  0.1000025
abs_delta(timer, definitive dt=100ms):  0.0000025
--------------------------------------------------------------
Measuring 1.0s in 0.1s steps in real-world time via the high-resolution process timer took 411.07519721984863 sec.

## Bringing it all together - B - Measure the Simulation Time

In the second stage, we're measuring the actual particle simulation time with the different timer means during process-concurrency interference. Our new simulation core looks then as such:

In [17]:
sim = Simulation(fX, fY, ftimes, fU, fV)
for i in range(0, 5000):
    pt = (b - a) * numpy.random.random_sample(2)  + a
    sim.add_particle(pt[0], pt[1])

nsync.clear()
concurrent_processes = [Process(target=sim_interference, args=(nsync,)) for i in range(0, 100)]
for proc in concurrent_processes:
    proc.start()

rt_global_lr_times_kernel = []
rt_global_hr_times_kernel = []
ptimer_hr_times_kernel = []

print("==== running the simulation with competing procs ====")
stime_rt_lr_sim = rt_timer()
stime_rt_hr_sim = rt_timer_hr()
stime_ptimer_hr_sim = compute_ptimer_hr()
prev_int_proc = -1
cur_int_proc = 0
while sim.sim_time < sim.sim_end_time:
    stime_rt_global_lr = rt_timer()
    stime_rt_global_hr = rt_timer_hr()
    stime_ptimer_hr = compute_ptimer_hr()
    sim.advect_once(dt)
    etime_rt_global_lr = rt_timer()
    etime_rt_global_hr = rt_timer_hr()
    etime_ptimer_hr = compute_ptimer_hr()
    rt_global_lr_times_kernel.append(etime_rt_global_lr-stime_rt_global_lr)
    rt_global_hr_times_kernel.append(etime_rt_global_hr-stime_rt_global_hr)
    ptimer_hr_times_kernel.append(etime_ptimer_hr-stime_ptimer_hr)
    proc_done = (sim.sim_time / sim.sim_end_time)*100
    cur_int_proc = int(math.floor(proc_done/10))
    if prev_int_proc != cur_int_proc:
        print("Processing at {:10.7f} percent ...".format(proc_done))
    prev_int_proc = cur_int_proc

nsync.set()
etime_rt_lr_sim = rt_timer()
etime_rt_hr_sim = rt_timer_hr()
etime_ptimer_hr_sim = compute_ptimer_hr()
print("Total real-time runtime (low-res timer): {:10.7f} sec.".format(etime_rt_lr_sim-stime_rt_lr_sim))
print("Total real-time runtime (high-res timer): {:10.7f} sec.".format(etime_rt_hr_sim-stime_rt_hr_sim))
print("Total processing time (high-res timer): {:10.7f} sec.".format(etime_ptimer_hr_sim-stime_ptimer_hr_sim))
avg_rt_lr = numpy.array(rt_global_lr_times_kernel).mean()
avg_rt_hr = numpy.array(rt_global_hr_times_kernel).mean()
avg_ptimer_hr = numpy.array(ptimer_hr_times_kernel).mean()
print("Average real-time kernel runtime (low-res timer): {:10.7f} sec.".format(avg_rt_lr))
print("Average real-time kernel runtime (high-res timer): {:10.7f} sec.".format(avg_rt_hr))
print("Average processing kernel time (high-res timer): {:10.7f} sec.".format(avg_ptimer_hr))


for proc in concurrent_processes:
    proc.join()
    
del sim

==== running the simulation with competing procs ====
Processing at  0.1351351 percent ...
Processing at 10.0000000 percent ...
Processing at 20.0000000 percent ...
Processing at 30.0000000 percent ...
Processing at 40.0000000 percent ...
Processing at 50.0000000 percent ...
Processing at 60.0000000 percent ...
Processing at 70.0000000 percent ...
Processing at 80.0000000 percent ...
Processing at 90.0000000 percent ...
Processing at 100.0000000 percent ...
Total real-time runtime (low-res timer): 793.3096852 sec.
Total real-time runtime (high-res timer): 793.3096816 sec.
Total processing time (high-res timer): 63.3586130 sec.
Average real-time kernel runtime (low-res timer):  1.0709121 sec.
Average real-time kernel runtime (high-res timer):  1.0709135 sec.
Average processing kernel time (high-res timer):  0.0855392 sec.


## Bringing it all together - C - I/O time vs. Compute time

In this last case, we want to now get a measure of the overall I/O time (for files and memory) and the bare compute time. This will, in the end, reliable tell us something about the simulation behaviour and where the bottlenecks are.

In [18]:
del fX
del fY
del ftimes
del fU
del fV
# ==== Load flow-field data - measure the time ==== #
s_io_file_time = compute_ptimer_hr()
f = netcdf.netcdf_file('perlin.nc', 'r')
ftimes = f.variables['time'].data
fX = f.variables['lon'].data
fY = f.variables['lat'].data
fU = f.variables['u'].data
fV = f.variables['v'].data
f.close()
e_io_file_time = compute_ptimer_hr()
io_file_time = e_io_file_time-s_io_file_time

# griddims = (ftimes, fY, fX)

/home/ckehl/.local/lib/python3.6/site-packages/scipy/io/netcdf.py:314: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)


simulation field data obtained prepared 

In [19]:
class Simulation_Benchmark(Simulation):
    def __init__(self, fx, fy, ft, fu, fv, num_visual_traces=0):
        super(Simulation_Benchmark, self).__init__(fx, fy, ft, fu, fv, num_visual_traces)
        self._io_mem_time = 0
        self._compute_time = 0
        
    def __del__(self):
        super(Simulation_Benchmark, self).__del__()
        
    def add_particle(self, x, y):
        stime = compute_ptimer_hr()
        super(Simulation_Benchmark, self).add_particle(x,y)
        etime = compute_ptimer_hr()
        self._io_mem_time += (etime-stime)
        
    def advect_once(self, dt):
        stime_calc = compute_ptimer_hr()
        fdt = self._ft[1]-self._ft[0]
        etime_calc = compute_ptimer_hr()
        self._compute_time += (etime_calc-stime_calc)
        x = []
        y = []
        t = []
        stime_mem = compute_ptimer_hr()
        for p in self._data:
            x.append(p.pt[0])
            y.append(p.pt[1])
            t.append(p.pt[2])
        pts = (numpy.array(t), numpy.array(y), numpy.array(x))
        etime_mem = compute_ptimer_hr()
        self._io_mem_time += (etime_mem-stime_mem)
        stime_calc = compute_ptimer_hr()
        us = interpn(self._gdims, self._fu, pts, method='linear', fill_value=.0)
        vs = interpn(self._gdims, self._fv, pts, method='linear', fill_value=.0)
        for i, p in enumerate(self._data):
            #p.advect(self._fu, self._fv, self._gdims, dt, fdt)
            p.advect_uv(us[i], vs[i], self._gdims, dt, fdt)
        self._simtime += dt
        etime_calc = compute_ptimer_hr()
        self._compute_time += (etime_calc-stime_calc)
    
    @property
    def io_mem_time(self):
        return self._io_mem_time
    
    @property
    def compute_time(self):
        return self._compute_time

Now run the simulation and measure the total and partial times

In [20]:
sim = Simulation_Benchmark(fX, fY, ftimes, fU, fV)
for i in range(0, 5000):
    pt = (b - a) * numpy.random.random_sample(2)  + a
    sim.add_particle(pt[0], pt[1])

ptimer_hr_times_kernel.clear()
print("==== running the simulation with competing procs ====")
stime_ptimer_hr_sim = compute_ptimer_hr()
prev_int_proc = -1
cur_int_proc = 0
while sim.sim_time < sim.sim_end_time:
    stime_ptimer_hr = compute_ptimer_hr()
    sim.advect_once(dt)
    etime_ptimer_hr = compute_ptimer_hr()
    ptimer_hr_times_kernel.append(etime_ptimer_hr-stime_ptimer_hr)
    proc_done = (sim.sim_time / sim.sim_end_time)*100
    cur_int_proc = int(math.floor(proc_done/10))
    if prev_int_proc != cur_int_proc:
        print("Processing at {:10.7f} percent ...".format(proc_done))
    prev_int_proc = cur_int_proc

etime_ptimer_hr_sim = compute_ptimer_hr()
sim_total_time = etime_ptimer_hr_sim-stime_ptimer_hr_sim
print("Total processing time (high-res timer): {:10.7f} sec.".format(sim_total_time))
avg_ptimer_hr = numpy.array(ptimer_hr_times_kernel).mean()
print("Average processing kernel time (high-res timer): {:10.7f} sec.".format(avg_ptimer_hr))

io_to_total_ratio = sim.io_mem_time / sim_total_time
compute_to_total_ratio = sim.compute_time / sim_total_time
compute_to_iototal = sim.compute_time / (sim.io_mem_time+io_file_time)

print("Compute percentage of simulation time: {:10.7f}".format(compute_to_total_ratio*100.0))
print("I/O percentage of simulation time: {:10.7f}".format(io_to_total_ratio*100.0))
print("Ratio Compute time vs. I/O time: {:10.7f}".format(compute_to_iototal))

del sim

==== running the simulation with competing procs ====
Processing at  0.1351351 percent ...
Processing at 10.0000000 percent ...
Processing at 20.0000000 percent ...
Processing at 30.0000000 percent ...
Processing at 40.0000000 percent ...
Processing at 50.0000000 percent ...
Processing at 60.0000000 percent ...
Processing at 70.0000000 percent ...
Processing at 80.0000000 percent ...
Processing at 90.0000000 percent ...
Processing at 100.0000000 percent ...
Total processing time (high-res timer): 26.8487899 sec.
Average processing kernel time (high-res timer):  0.0362425 sec.
Compute percentage of simulation time: 83.8656838
I/O percentage of simulation time: 13.7216521
Ratio Compute time vs. I/O time:  6.1090806


The final output number here is the key figure. Conclusion that are made about it:
- *compute vs. io* >> 1.0: high computation density - lots of benefit from C-kernel computation or GPU Computing
- *compute vs. io* >= 1.0: if I/O and CPU load are balanced, improved memory access procedures are the first-most advice to improve performance
- 1.0 < *compute vs. io* < 0.1: main bottleneck culprit is memory - likely many *copy-array1-to-array2* operations, of which some may be unnecessary ? Improve memory utilisation by avoiding unnecessary copies
- *compute vs. io* < 0.1 **or** *compute vs. io* << 0.1: main bottleneck (due to the orders of magnitude difference) has to be file-, database- or network communication. The code needs to optimize file access, only load required subset data and also buffer-write results into files. **Investing time in C-kernels, GPU Computing, Vectorisation or similar compute-enhancing techniques is a waste of time for simulations in this bracket!!**
